In [3]:
import os
os.chdir('..')

In [4]:
pwd

'/mnt/md1/user_victor/automatic_melody_harmonization'

In [5]:
import numpy as np
import pickle
from constants import Constants
from tqdm import tqdm

In [6]:
symbol_and_indices = np.load('./data/chord_indices.npy')
chord_onehot = np.load('./data/chord_onehot.npy')
chord_indices_96 = np.load('./data/number_96.npy')
chord_onehot_96 = np.load('./data/onehot_96.npy')

In [14]:
i = 3
symbol_and_indices[i][:10]

array([[  0],
       [  0],
       [234],
       [234],
       [107],
       [107],
       [234],
       [234],
       [107],
       [107]])

In [16]:
chord_onehot[i][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [17]:
chord_indices_96[i][:10]

array([[ 0],
       [ 0],
       [40],
       [40],
       [ 0],
       [ 0],
       [40],
       [40],
       [ 0],
       [ 0]])

In [18]:
chord_onehot_96[i][0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# symbol_and_indices[1]

In [12]:
# Load chord symbol data
f = open('./data/symbol_data_2_beat', 'rb')
symbol_data = pickle.load(f)
f.close()
print('Chord symbol data length:',len(symbol_data))

chord_groundtruth_pianoroll = np.load('./data/chord_groundtruth.npy')
print('Chord_groundtruth_pianoroll:',chord_groundtruth_pianoroll.shape)

chord_lengths = np.load('./data/length.npy')
print('Chord length:',chord_lengths.shape)

Chord symbol data length: 17430
Chord_groundtruth_pianoroll: (17430, 272, 128)
Chord length: (17430,)


In [14]:
# List all chords 
all_chords = []
for chord in symbol_data:
    all_chords += chord

all_chords = list(set(all_chords))
# try:
# #     all_chords.remove(None)
# #     all_chords.remove('')
# except:
#     pass

all_chords = sorted(all_chords)

In [212]:
print('Total chord numbers:',len(all_chords))

Total chord numbers: 633


In [215]:
all_chords[:10]

['',
 'A',
 'A (add9)',
 'A sus2',
 'A sus4',
 'A sus42',
 'A | Db',
 'A | Db (#5)',
 'A | E',
 'A | E (add9)']

In [20]:
# Chord symbol to indices
# Pad 0 to the positions if the length of sequence is smaller than max length 
symbol_and_indices = dict(zip(all_chords,[i for i in range(len(all_chords))]))
indices_and_symbol = dict(zip([i for i in range(len(all_chords))],all_chords))
## Save symbol and fingerring pairs
f = open('./data/indices_to_symbol' , 'wb')
pickle.dump(indices_and_symbol, f)
f.close()

chord_indices = []
for symbol in symbol_data:
    chord_index = [symbol_and_indices[s] if s is not '' else 0 for s in symbol]
    chord_indices.append(np.asarray(chord_index))

for i in range(len(chord_indices)):
#     print(i)
    chord_indices[i] = np.pad(chord_indices[i], (0, Constants.MAX_SEQUENCE_LENGTH - chord_indices[i].shape[0]), constant_values = 0)
    
chord_indices = np.expand_dims(np.asarray(chord_indices),-1)
chord_indices = chord_indices.astype(int)

In [219]:
# Chord indices to onehot
chord_onehot = np.zeros((chord_indices.shape[0],Constants.MAX_SEQUENCE_LENGTH,len(all_chords)))

for i in range(chord_indices.shape[0]):
    for t in range(chord_lengths[i]):
        if chord_indices[i][t][0] != -1 and chord_indices[i][t][0] != 0:
            chord_onehot[i][t][chord_indices[i][t][0]] = 1

In [220]:
np.argmax(chord_onehot[1][:10],axis=-1)

array([  0,   0, 234, 234, 107, 107, 234, 234, 107, 107])

In [162]:
chord_onehot.shape

(17430, 272, 632)

In [221]:
## Save data
np.save('./data/chord_indices', chord_indices)
np.save('./data/chord_onehot', chord_onehot)

In [222]:
song_index = 115
symbol = symbol_data[song_index]
pianoroll = chord_groundtruth_pianoroll[song_index][:chord_lengths[song_index]]

print('symbol',symbol)
print('symbol shape',len(symbol))
print('pianoroll',pianoroll)
print('pianoroll shape ',pianoroll.shape)

symbol ['am', '', '', '']
symbol shape 4
pianoroll [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 100   0   0 100   0   0   0 100   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0 

In [223]:
## Collect and find the most common ('chord symbol',fingerring) pair data
chordsymbol2pianoroll = []
for song_index in tqdm(range(len(symbol_data))):
    symbol = symbol_data[song_index]
    length = len(symbol)
    pianoroll = chord_groundtruth_pianoroll[song_index][:length]
    data = [[s,p] for s, p in zip(symbol,pianoroll)]
    unique_index = sorted(np.unique(pianoroll,return_index = True,axis=0)[-1])
    for ui in unique_index:
        if unique_index[-1] < len(symbol) and (len(symbol) <= chord_lengths[song_index] + 1 or len(symbol) >= chord_lengths[song_index] - 1): 
            chordsymbol2pianoroll.append(data[ui]) 
        else:
            print('mismatch',song_index)
            break
            
# Find most frequently chords fingerring
uniq_fingerring = []
for target_chord_symbol in all_chords:
    duplicate = []
    for i in chordsymbol2pianoroll:
        if i[0] == target_chord_symbol:
            duplicate.append(i[1])

    duplicate = np.asarray(duplicate)    
    uniq, indices, counts = np.unique(duplicate,return_index =True,return_counts = True,axis=0)
    uniq_fingerring.append(uniq[counts.argmax(axis=-1)])

symbol_and_fingerring = dict(zip(all_chords,uniq_fingerring))
## Save symbol and fingerring pairs
f = open('./data/symbol_and_fingerring' , 'wb')
pickle.dump(symbol_and_fingerring, f)
f.close()


100%|██████████| 17430/17430 [00:12<00:00, 1428.81it/s]
